## How to Modify This Notebook

This notebook is completely self-contained and can be easily modified:

1. **Change the data**: Modify the data generation logic in the "Sample Data Setup" section to test different scenarios
2. **Adjust metrics**: Add new metrics to the `compute_metrics()` function
3. **Test different ratios**: Change the fusion/fission rates or error rates in the data generation
4. **Add visualizations**: Use matplotlib or seaborn to create charts from the metrics

### Key Parameters to Experiment With:
- Number of examples (currently 200 for each method)
- Fusion/fission decision ratios
- Error rates for each method
- API call costs (currently fusion=1, fission=2)

The notebook will automatically recalculate all metrics when you modify the input data!

In [ ]:
# Display detailed results
print("=" * 50)
print("DKW CONTROLLER EVALUATION RESULTS")
print("=" * 50)

print(f"\n📊 BASELINE METHOD:")
print(f"   Fusion Rate: {metrics['baseline']['fusion_rate']:.1%}")
print(f"   Fission Rate: {metrics['baseline']['fission_rate']:.1%}")
print(f"   Error Rate: {metrics['baseline']['error_rate']:.1%}")
print(f"   Total API Calls: {metrics['baseline']['api_calls']}")
print(f"   Avg Calls/Example: {metrics['baseline']['avg_calls_per_example']:.2f}")

print(f"\n🚀 PROPOSED METHOD:")
print(f"   Fusion Rate: {metrics['proposed']['fusion_rate']:.1%}")
print(f"   Fission Rate: {metrics['proposed']['fission_rate']:.1%}")
print(f"   Error Rate: {metrics['proposed']['error_rate']:.1%}")
print(f"   Total API Calls: {metrics['proposed']['api_calls']}")
print(f"   Avg Calls/Example: {metrics['proposed']['avg_calls_per_example']:.2f}")

print(f"\n✨ IMPROVEMENT SUMMARY:")
print(f"   API Reduction: {metrics['improvement']['api_reduction_pct']:.1f}%")
print(f"   Error Rate Change: {metrics['improvement']['error_rate_diff']:+.1%}")

# Main result as in original script
print(f"\n🎯 KEY RESULT: API reduction: {metrics['improvement']['api_reduction_pct']:.1f}%")

print("\n" + "=" * 50)
print("COMPLETE JSON OUTPUT:")
print("=" * 50)
print(eval_output)

## Results Display

Pretty-print the evaluation results with detailed breakdown.

In [ ]:
# Compute metrics using our sample data
metrics = compute_metrics(results)

# Save results (this replaces writing to "eval_out.json")
eval_output = json.dumps(metrics, indent=2)
print("Evaluation completed!")
print("\nResults saved (would be written to eval_out.json):")

## Run Evaluation

Compute the metrics for both baseline and proposed methods.

In [ ]:
def compute_metrics(results: dict) -> dict:
    """Compute evaluation metrics."""
    metrics = {}

    for method in ["baseline", "proposed"]:
        preds = results[method]

        # Count decisions
        fusion_count = sum(1 for p in preds if p["decision"] == "fusion")
        fission_count = sum(1 for p in preds if p["decision"] == "fission")

        # Compute error rate
        errors = sum(1 for p in preds if p["error"])
        error_rate = errors / len(preds)

        # API calls (fusion=1, fission=2)
        api_calls = fusion_count + 2 * fission_count

        metrics[method] = {
            "fusion_rate": fusion_count / len(preds),
            "fission_rate": fission_count / len(preds),
            "error_rate": error_rate,
            "api_calls": api_calls,
            "avg_calls_per_example": api_calls / len(preds),
        }

    # Compute improvement
    baseline_calls = metrics["baseline"]["avg_calls_per_example"]
    proposed_calls = metrics["proposed"]["avg_calls_per_example"]
    metrics["improvement"] = {
        "api_reduction_pct": (baseline_calls - proposed_calls) / baseline_calls * 100,
        "error_rate_diff": metrics["proposed"]["error_rate"] - metrics["baseline"]["error_rate"],
    }

    return metrics

print("Metrics computation function defined!")

## Metrics Computation Function

The core evaluation function that computes performance metrics for both methods.

In [ ]:
# Create sample data that matches the expected results
# This replaces reading from "../experiment_001/method_out.json"

# Generate baseline results: 200 examples, all fission, 8% error rate
baseline_preds = []
for i in range(200):
    is_error = i < 16  # First 16 examples have errors (8% of 200)
    baseline_preds.append({
        "decision": "fission",  # Baseline always chooses fission
        "error": is_error
    })

# Generate proposed results: 200 examples, 65% fusion, 35% fission, 9% error rate
proposed_preds = []
for i in range(200):
    is_error = i < 18  # First 18 examples have errors (9% of 200)
    decision = "fusion" if i < 130 else "fission"  # 130 fusion (65%), 70 fission (35%)
    proposed_preds.append({
        "decision": decision,
        "error": is_error
    })

# Create the results dictionary that would have been loaded from JSON
results = {
    "baseline": baseline_preds,
    "proposed": proposed_preds
}

print(f"Generated data:")
print(f"- Baseline: {len(results['baseline'])} predictions")
print(f"- Proposed: {len(results['proposed'])} predictions")
print(f"- Baseline decisions: {sum(1 for p in baseline_preds if p['decision'] == 'fission')} fission, {sum(1 for p in baseline_preds if p['decision'] == 'fusion')} fusion")
print(f"- Proposed decisions: {sum(1 for p in proposed_preds if p['decision'] == 'fission')} fission, {sum(1 for p in proposed_preds if p['decision'] == 'fusion')} fusion")

## Sample Data Setup

Instead of reading from external JSON files, we'll create inline sample data that represents the experimental results. This data simulates:
- **Baseline method**: Always chooses fission (more expensive, 2 API calls per decision)
- **Proposed method**: Intelligently chooses between fusion (1 call) and fission (2 calls)

In [ ]:
"""Evaluation script for DKW Controller."""
import json
import numpy as np

print("Libraries imported successfully!")

# DKW Controller Evaluation

This notebook evaluates the performance of a DKW (Decision-Knowledge-Workflow) controller by comparing baseline and proposed methods. The evaluation focuses on:
- **Fusion vs Fission decisions**: Different strategies for handling requests
- **Error rates**: Frequency of incorrect predictions
- **API call efficiency**: Number of API calls required per example

## Metrics Computed
- Fusion/Fission rates for each method
- Error rates comparison
- API call reduction percentage
- Average calls per example